# Applications de DIMSUM

In [ ]:
import numpy as np
import scipy as sc
import scipy.sparse as sp
import random
from collections import defaultdict
from pyspark.sql import SparkSession
from pyspark.mllib.linalg.distributed import MatrixEntry, CoordinateMatrix
from numpy import linalg as LA
from collections import Counter

## Régression linéaire en "haute dimension" (nombre de lignes >>> nombre de colonnes)

### Génération des données 

Nous allons générer des données de poids en fonction de la taille^2 avec un IMC=25 avec des données bruités

$ poids(kg)= (IMC=25)*taille^2(m)+ \sum^n_{i=1} Bruit_i$

In [ ]:
def generate_data(number_of_observation:int,beta:int=25):
    poids=np.random.uniform(10,100, number_of_observation)
    taille_carre=poids/25
    return poids,taille_carre

In [ ]:
generate_data(int(1e6))